In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [4]:
ls "."

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: BCA4-98AF

 C:\Users\inhoinno\DACON\2020_DACON-Kaggle\소설저자AI 디렉터리

2020-12-04  오후 04:48    <DIR>          .
2020-12-04  오후 04:48    <DIR>          ..
2020-12-04  오후 03:32    <DIR>          .ipynb_checkpoints
2020-12-04  오후 02:55            58,355 13-lr-tfidf.ipynb
2020-11-26  오후 10:42            75,612 15_lstm_glove_inhoinho_v0_1.ipynb
2020-12-02  오후 06:12            76,044 15_lstm_glove_inhoinho_v0_2_ipynb의_사본.ipynb
2020-11-26  오후 08:29            38,607 15-lstm-glove-inho.ipynb
2020-12-04  오후 03:29           101,038 16-attention.ipynb
2020-11-24  오전 10:49            33,874 Augmentation.ipynb
2020-12-01  오후 04:01    <DIR>          build
2020-12-02  오후 09:19           318,220 CNN_CV_shared.ipynb
2020-12-04  오전 01:10            98,597 CNN_CV_shared-Refactoring.ipynb
2020-11-24  오전 11:15    <DIR>          dataset
2020-11-26  오후 08:29    <DIR>          dku-kaggle-class
2020-12-04  오전 06:00           113,621 DNN_CV_shared-Refactoring.ipynb
2020-12-01  오후 04:01  

In [5]:
data_dir = Path('./open')
trn_file = data_dir / 'train_processed_500.csv'
tst_file = data_dir / 'test_x.csv'
submission = pd.read_csv("./open/sample_submission.csv")

val_dir = Path('./ensemble/build/val')
tst_dir = Path('./ensemble/build/tst')
sub_dir = Path('./ensemble/build/sub')

algo_name = 'lgmbcv'
feature_name = 'meta'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
seed = 42


In [6]:
train = pd.read_csv(trn_file)
test = pd.read_csv(tst_file)
feature= pd.concat([train,test], axis =0)
feature.fillna(-1, inplace=True)
print(train.info())
print(test.info())
print(train.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183468 entries, 0 to 183467
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   index   183468 non-null  int64 
 1   text    183468 non-null  object
 2   author  183468 non-null  int64 
 3   length  183468 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 5.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19617 entries, 0 to 19616
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   19617 non-null  int64 
 1   text    19617 non-null  object
dtypes: int64(1), object(1)
memory usage: 306.6+ KB
None
(183468, 4)


In [7]:
train['length'] = train['text'].apply(len)
idx_under_20 = train[train['length'] < 20].index
train = train.drop(idx_under_20)
train = train.reset_index()
train.describe()

,level_0,index,author,length
count,181013.0000,181013.0000,181013.0000,181013.0000
mean,91514.5870,91514.5870,2.0491,164.5379
std,53025.4182,53025.4182,1.2863,178.6952
min,0.0000,0.0000,0.0000,20.0000
25%,45490.0000,45490.0000,1.0000,65.0000
50%,91441.0000,91441.0000,2.0000,113.0000
75%,137458.0000,137458.0000,3.0000,189.0000
max,183467.0000,183467.0000,4.0000,2454.0000


In [8]:
df = train.drop_duplicates(['text'])
df = df.drop(columns = ['level_0'])
df = df.reset_index()
df['index'] = df['index'].reset_index()
train = df
train= train.drop(columns=['level_0'])
train

,index,text,author,length
0,0,He was almost choking. There was so much so mu...,3,235
1,1,Your sister asked for it I suppose,2,34
2,2,She was engaged one day as she walked in peru...,1,312
3,3,The captain was in the porch keeping himself c...,4,305
4,4,Have mercy gentlemen odin flung up his hands. ...,3,215
...,...,...,...,...
95501,95501,In the diningroom they were soon joined by od...,1,150
95502,95502,One came from her books and the other from he...,1,56
95503,95503,The faces of both however were tolerably calm...,1,252
95504,95504,Still the ticking in the hall,2,29


<h3>Stacking Feature 생성<h3>

In [10]:
model_names = ['cnn_glove','cnn_glove-lrs']
trn = []
tst = []
feature_names = []
for model in model_names:
    trn.append(np.loadtxt(val_dir / f'{model}.val.csv', delimiter=','))
    tst.append(np.loadtxt(tst_dir / f'{model}.tst.csv', delimiter=','))
    feature_names += [f'{model}_class0', f'{model}_class1', f'{model}_class2']
    
trn = np.hstack(trn)
tst = np.hstack(tst)
feature_names

['cnn_glove_class0',
 'cnn_glove_class1',
 'cnn_glove_class2',
 'cnn_glove-lrs_class0',
 'cnn_glove-lrs_class1',
 'cnn_glove-lrs_class2']

In [13]:
y = train.author.values
y.shape

(95506,)

In [14]:
n_fold = 5
n_class =5
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


<h2> LGBM 모델학습 </h2>

In [15]:
trn

array([[1.40000e-05, 0.00000e+00, 9.00000e-06, ..., 2.00000e-06,
        9.99985e-01, 0.00000e+00],
       [3.67319e-01, 2.15480e-01, 1.45798e-01, ..., 7.39350e-02,
        1.27135e-01, 1.68920e-01],
       [1.38380e-02, 9.86112e-01, 1.30000e-05, ..., 0.00000e+00,
        0.00000e+00, 1.00000e-06],
       ...,
       [2.74957e-01, 1.65132e-01, 2.26131e-01, ..., 1.93915e-01,
        3.24310e-02, 5.11213e-01],
       [5.45610e-02, 9.21954e-01, 8.37900e-03, ..., 1.11100e-03,
        9.16000e-04, 1.72400e-03],
       [2.83360e-02, 9.64644e-01, 2.49900e-03, ..., 2.15000e-04,
        3.59900e-03, 1.63800e-02]])

In [16]:
params = {
    "objective": "multi_logloss",
    "n_estimators": 1500,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.2)),
    "num_leaves": hp.choice("num_leaves", [10,16,24,32,64]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, 1.0, 0.1),
    "subsample": hp.quniform("subsample", .8, 1.0, 0.05),
    "min_child_samples": hp.choice('min_child_samples', np.linspace(10, 200, 5, dtype=int)),
    
}

In [17]:
def objective(hyperparams):
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'training model for CV #{i}')
        model = lgb.LGBMClassifier(**params, **hyperparams)
        model.fit(trn[i_trn], y[i_trn],
                  eval_set=[(trn[i_val], y[i_val])],
              eval_metric="multi_logloss",
              early_stopping_rounds=30,
              verbose=False)
        score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

job exception: Found input variables with inconsistent numbers of samples: [87288, 95506]



  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]


ValueError: Found input variables with inconsistent numbers of samples: [87288, 95506]

In [ ]:
import lightgbm as lgb
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=30)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

In [ ]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
imp = pd.DataFrame({'feature': feature_names, 'importance': clf.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh')

Submission 생성 및 제출

In [ ]:
sub = submission
sub.head()

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv('LGBM_Stacking_withhyperopt_93,64.csv', index = False , encoding = 'utf-8-sig')

In [ ]:
import lightgbm as lgb

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=1000,
                             num_leaves=64,
                             learning_rate=0.1,
                             min_child_samples=10,
                             subsample=.5,
                             subsample_freq=1,
                             colsample_bytree=.8,
                             random_state=seed,
                             n_jobs=-1)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

In [ ]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
print(p_val.shape, p_tst.shape)


In [ ]:
#
# lgbm Stacking -> 93.64% 
# cv Xgboost -> 93.628 % 
#   XGB보다Stacking가 성능이 더 좋은 느낌
#